In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['CPE']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('cytopathic effect', 679),
 ('carbapenemase producing enterobacteriaceae', 301),
 ('clostridium perfringens enterotoxin', 109),
 ('carboxypeptidase e', 99),
 ('cytoplasmic polyadenylation element', 72),
 ('cytopathogenic effect', 44),
 ('c perfringens enterotoxin', 30),
 ('choroid plexus epithelium', 20),
 ('carbapenemase producing enterobacterales', 16),
 ('caspofungin paradoxical effect', 15),
 ('crude peanut extract', 14),
 ('cns penetration effect', 13),
 ('choroid plexus epithelial', 9),
 ('cardiogenic pulmonary edema', 8),
 ('ceramide phosphoethanolamine', 8),
 ('cocoa polyphenol extract', 8),
 ('carbon paste electrode', 7),
 ('complicated parapneumonic effusions', 7),
 ('cloud point extract', 7),
 ('coffee polyphenol extract', 7),
 ('chromosomal positive effect', 6),
 ('carp pituitary extract', 5),
 ('cinnamon polyphenol extract', 5),
 ('cocoa phenolic extract', 5),
 ('central nervous system penetration effect', 5),
 ('clostridium perfringens', 4),
 ('control potential electro

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-02 00:14:23] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'c perfringens enterotoxin': 'MESH:C035598',
  'carbapenemase producing enterobacterales': 'MESH:D000073182',
  'carbapenemase producing enterobacteriaceae': 'MESH:D000073182',
  'carbon paste electrode': 'MESH:D004566',
  'carboxypeptidase e': 'HGNC:2303',
  'cardiogenic pulmonary edema': 'DOID:DOID:11396',
  'carp pituitary extract': 'ungrounded',
  'caspofungin paradoxical effect': 'caspofungin_paradoxical_effect',
  'central nervous system penetration effect': 'cns_penetration',
  'ceramide phosphoethanolamine': 'CHEBI:CHEBI:73204',
  'chloroplastic pigment equivalent': 'ungrounded',
  'choroid plexus epithelial': 'NCIT:C42079',
  'choroid plexus epithelium': 'NCIT:C42079',
  'chromosomal positive effect': 'MESH:D055012',
  'cinnamon polyphenol extract': 'ungrounded',
  'citrus peel extract': 'ungrounded',
  'clostridium perfringens': 'MESH:C035598',
  'clostridium perfringens enterotoxin': 'MESH:C035598',
  'cloud point extract': 'ungrounded',
  'cns penetration effect': 'cns_pe

In [10]:
grounding_map, names, pos_labels = [{'c perfringens enterotoxin': 'MESH:C035598',
  'carbapenemase producing enterobacterales': 'MESH:D000073182',
  'carbapenemase producing enterobacteriaceae': 'MESH:D000073182',
  'carbon paste electrode': 'MESH:D004566',
  'carboxypeptidase e': 'HGNC:2303',
  'cardiogenic pulmonary edema': 'DOID:DOID:11396',
  'carp pituitary extract': 'ungrounded',
  'caspofungin paradoxical effect': 'caspofungin_paradoxical_effect',
  'central nervous system penetration effect': 'cns_penetration',
  'ceramide phosphoethanolamine': 'CHEBI:CHEBI:73204',
  'chloroplastic pigment equivalent': 'ungrounded',
  'choroid plexus epithelial': 'NCIT:C42079',
  'choroid plexus epithelium': 'NCIT:C42079',
  'chromosomal positive effect': 'MESH:D055012',
  'cinnamon polyphenol extract': 'ungrounded',
  'citrus peel extract': 'ungrounded',
  'clostridium perfringens': 'MESH:C035598',
  'clostridium perfringens enterotoxin': 'MESH:C035598',
  'cloud point extract': 'ungrounded',
  'cns penetration effect': 'cns_penetration',
  'cocoa phenolic extract': 'NCIT:C121851',
  'cocoa polyphenol extract': 'NCIT:C121851',
  'cocoa proanthocyanidin extract': 'ungrounded',
  'coffee polyphenol extract': 'NCIT:C154289',
  'complicated parapneumonic effusions': 'MESH:D010996',
  'composite polymer electrolyte': 'CHEBI:CHEBI:53369',
  'conjugated polyelectrolyte': 'CHEBI:CHEBI:53369',
  'constant phase element': 'ungrounded',
  'control potential electrolysis': 'ungrounded',
  'conventional polyethylene': 'CHEBI:CHEBI:53227',
  'crude peanut extract': 'ungrounded',
  'cryptolepine': 'CHEBI:CHEBI:3930',
  'cytopathic effect': 'MESH:D003588',
  'cytopathogenic effect': 'MESH:D003588',
  'cytoplasmic polyadenylation element': 'cytoplasmic_polyadenylation_element'},
 {'MESH:C035598': 'enterotoxin, Clostridium',
  'MESH:D000073182': 'Carbapenem-Resistant Enterobacteriaceae',
  'MESH:D004566': 'Electrodes',
  'HGNC:2303': 'CPE',
  'DOID:DOID:11396': 'pulmonary edema',
  'caspofungin_paradoxical_effect': 'caspofungin_paradoxical_effect',
  'cns_penetration': 'cns_penetration',
  'CHEBI:CHEBI:73204': 'ceramide phosphoethanolamine',
  'NCIT:C42079': 'Choroid Plexus Epithelium',
  'MESH:D055012': 'Chromosomal Position Effects',
  'NCIT:C121851': 'Cocoa Extract',
  'NCIT:C154289': 'Coffea arabica Extract',
  'MESH:D010996': 'Pleural Effusion',
  'CHEBI:CHEBI:53369': 'polyelectrolyte macromolecule',
  'CHEBI:CHEBI:53227': 'poly(ethylene)',
  'CHEBI:CHEBI:3930': 'cryptolepine',
  'MESH:D003588': 'Cytopathogenic Effect, Viral',
  'cytoplasmic_polyadenylation_element': 'cytoplasmic_polyadenylation_element'},
 ['HGNC:2303',
  'MESH:C035598',
  'MESH:D000073182',
  'MESH:D003588',
  'NCIT:C42079']]

In [11]:
excluded_longforms = []

In [12]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [22]:
additional_entities = {}

In [23]:
unambiguous_agent_texts = {}

In [24]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [25]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [26]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [27]:
intersection1

[('HGNC:2303', 'HGNC:2303', 154)]

In [32]:
corpus[-10:]

[('Glioblastoma (GBM), the most frequent and aggressive malignant primary brain tumor, is characterized by a highly invasive growth. In our previous study we showed that overexpression of Carboxypeptidase E (CPE) mitigated glioma cell migration. In the present study we aimed at deciphering the regulatory mechanisms of the secreted form of CPE (sCPE). By transcriptome analysis and inhibition of signaling pathways involved in the regulation of cell growth and motility, we discovered that overexpression of sCPE was accompanied by differential regulation of mRNAs connected to the motility-associated networks, among others FAK, PAK, Cdc42, integrin, STAT3 as well as TGF-β. Especially SLUG was downregulated in sCPE-overexpressing glioma cells, paralleled by reduced expression of matrix-metalloproteinases (MMP) and, in consequence, by decreased cell migration. Expression of SLUG was regulated by ERK since inhibition of ERK reverted sCPE-mediated SLUG downregulation and enhanced cell motility.

In [28]:
intersection2

[('MESH:D003588', 'HGNC:2303', 0),
 ('HGNC:2303', 'HGNC:2303', 33),
 ('cytoplasmic_polyadenylation_element', 'HGNC:2303', 0),
 ('MESH:C035598', 'HGNC:2303', 0),
 ('MESH:D010996', 'HGNC:2303', 0),
 ('ungrounded', 'HGNC:2303', 0),
 ('MESH:D055012', 'HGNC:2303', 0),
 ('MESH:D004566', 'HGNC:2303', 0),
 ('DOID:DOID:11396', 'HGNC:2303', 0),
 ('NCIT:C42079', 'HGNC:2303', 0),
 ('CHEBI:CHEBI:73204', 'HGNC:2303', 0),
 ('NCIT:C121851', 'HGNC:2303', 0),
 ('MESH:D000073182', 'HGNC:2303', 0),
 ('cns_penetration', 'HGNC:2303', 0),
 ('CHEBI:CHEBI:53227', 'HGNC:2303', 0),
 ('CHEBI:CHEBI:53369', 'HGNC:2303', 0),
 ('NCIT:C154289', 'HGNC:2303', 0),
 ('CHEBI:CHEBI:3930', 'HGNC:2303', 0),
 ('caspofungin_paradoxical_effect', 'HGNC:2303', 0)]

In [30]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['CPE'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [33]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-02 01:08:31] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-02 01:09:04] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.955588974480681 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [17]:
classifier.stats

{'label_distribution': {'MESH:D003588': 557,
  'HGNC:2303': 75,
  'cytoplasmic_polyadenylation_element': 62,
  'MESH:C035598': 92,
  'MESH:D010996': 3,
  'ungrounded': 36,
  'MESH:D055012': 4,
  'MESH:D004566': 6,
  'DOID:DOID:11396': 5,
  'NCIT:C42079': 17,
  'CHEBI:CHEBI:73204': 6,
  'NCIT:C121851': 10,
  'MESH:D000073182': 261,
  'cns_penetration': 9,
  'CHEBI:CHEBI:53227': 2,
  'CHEBI:CHEBI:53369': 5,
  'NCIT:C154289': 1,
  'CHEBI:CHEBI:3930': 1,
  'caspofungin_paradoxical_effect': 4},
 'f1': {'mean': 0.965416, 'std': 0.012584},
 'precision': {'mean': 0.954414, 'std': 0.013735},
 'recall': {'mean': 0.97901, 'std': 0.010233},
 'caspofungin_paradoxical_effect': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'MESH:C035598': {'f1': {'mean': 0.955839, 'std': 0.021319},
  'pr': {'mean': 0.935088, 'std': 0.020207},
  'rc': {'mean': 0.977778, 'std': 0.027217}},
 'CHEBI:CHEBI:3930': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean':

In [19]:
disamb.dump(model_name, results_path)

In [36]:
print(disamb.info())

Disambiguation model for CPE

Produces the disambiguations:
	CPE*	HGNC:2303
	Carbapenem-Resistant Enterobacteriaceae*	MESH:D000073182
	Choroid Plexus Epithelium*	NCIT:C42079
	Chromosomal Position Effects	MESH:D055012
	Cocoa Extract	NCIT:C121851
	Coffea arabica Extract	NCIT:C154289
	Cytopathogenic Effect, Viral*	MESH:D003588
	Electrodes	MESH:D004566
	Pleural Effusion	MESH:D010996
	caspofungin_paradoxical_effect	caspofungin_paradoxical_effect
	ceramide phosphoethanolamine	CHEBI:CHEBI:73204
	cns_penetration	cns_penetration
	cryptolepine	CHEBI:CHEBI:3930
	cytoplasmic_polyadenylation_element	cytoplasmic_polyadenylation_element
	enterotoxin, Clostridium*	MESH:C035598
	poly(ethylene)	CHEBI:CHEBI:53227
	polyelectrolyte macromolecule	CHEBI:CHEBI:53369
	pulmonary edema	DOID:DOID:11396

Class level metrics:
--------------------
Grounding                              	Count	F1     
           Cytopathogenic Effect, Viral*	557	 0.9695
Carbapenem-Resistant Enterobacteriaceae*	261	0.98296
           

In [37]:
model_to_s3(disamb)